In [1]:
import requests
import json

import pandas as pd
import pandasql as ps
import numpy as np

from highcharts import Highchart

In [2]:
link = 'https://api.opendota.com/api/proMatches'

In [3]:
r = requests.get(link)
data = json.loads(r.text)

In [4]:
df = pd.DataFrame(data)

In [5]:
kyiv_major_matches = df[df['leagueid'] == 12964]

In [6]:
hero_stats = pd.DataFrame()

In [7]:
for match_id in kyiv_major_matches['match_id'].tolist():
    match_link = 'https://api.opendota.com/api/matches/{}'.format(match_id)
    r = requests.get(match_link)
    data = json.loads(r.text)
    match_df = pd.DataFrame(data['draft_timings'])
    match_df['match_id'] = match_id
    
    hero_stats = hero_stats.append(match_df)
    hero_stats.reset_index(drop = True)

C:\Users\Alec\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [8]:
heroes_link = 'https://api.opendota.com/api/heroes'

In [9]:
r = requests.get(heroes_link)
data = json.loads(r.text)
heroes_df = pd.DataFrame(data)
heroes = heroes_df[['id', 'localized_name', 'primary_attr']]

In [10]:
sql_query = """
SELECT  hero_stats.hero_id,
        heroes.localized_name AS hero_name,
        SUM(CASE 
            WHEN hero_stats.pick = 1 THEN 1
            ELSE 0
        END) AS hero_picks,
        SUM(CASE 
            WHEN hero_stats.pick = 0 THEN 1
            ELSE 0
        END) AS hero_bans
FROM hero_stats
JOIN heroes
ON hero_stats.hero_id = heroes.id
GROUP BY hero_id, hero_name
"""

hero_counts = ps.sqldf(sql_query)

In [11]:
hero_counts['total_pbs'] = hero_counts['hero_picks'] + hero_counts['hero_bans']
hero_counts = hero_counts.astype({'hero_id': 'int32'})

In [12]:
for i in heroes[['id', 'localized_name']].iterrows():
    hero_id = i[1][0]
    hero_name = i[1][1]
    
    if hero_name not in hero_counts['hero_name'].tolist():
        hero_counts = hero_counts.append({'hero_id': hero_id, 'hero_name': hero_name, 'hero_picks': 0, 'hero_bans': 0, 'total_pbs': 0}, ignore_index = True)
        
hero_counts.sort_values(by = ['total_pbs', 'hero_picks', 'hero_bans'], ascending = False, inplace = True)

In [13]:
# python Highchart documentation: https://github.com/kyper-data/python-highcharts
# javascript Highchart documentation: https://api.highcharts.com/highcharts/

# this instantiates the chart that we edit and add data to below
chart = Highchart()

options = {
    'title': {
      'text': 'Kyiv Major 2021 Wildcard + Group Stage Hero Stats'
    },
    
    'xAxis': {
        'type': 'category',
        'min': 0,
        'tickInterval': 1,
#         'categories': hero_counts['hero_name'].tolist(),
        'title': {
          'text': 'Hero'
        },
        'labels': {
          'step': 1,
          'rotation': -45,
          'animate': True
        }
    },

    'yAxis': [{ # Primary Axis
      'min': 0,
      'max': len(kyiv_major_matches) + 2,
      'title': { 
        'text': 'Number of Appearances',
      },
     'plotLines': 
        [{
          'id': 'Total Matches',
          'color': '#2a9cd1',
          'width': 2,
          'value': len(kyiv_major_matches),
          'dashStyle': 'dash',
          'zIndex': 2,
          'label': {
            'text': "Total Matches", 
            'x': 0, 
            'y': -10
          }
        }],
    }
    ],
#     const newSeries = chart.series[0].data.sort((a,b)=>a-b);
#     return chart.series[0].xAxis.categories;
#     'tooltip': {
#         'formatter': """
#         function() {
#             return this.series.options.dataSorting.enabled;
#         }"""
#     },
  
    'lang': {
        'thousandsSep': ','
    },

    'plotOptions': {
        'series': {
          'stacking': 'normal',
        	'events': {
            	'legendItemClick': """function(event) {
                	var s = chart.series;
                    for(i = 0; i < s.length; i++) {
                        console.log("-------------------------");
                        console.log("before: " + s[i].name + " - " + s[i].options.dataSorting.enabled);
                        if (this == s[i] && this.visible == true && s[1 - i].visible == true) {
                            s[i].setVisible(false);
                            s[i].update({dataSorting: {enabled: false, sortKey: 'y'}});
                            s[1 - i].update({dataSorting: {enabled: true, sortKey: 'y'}}, false);
                            s[1 - i].setData(s[1 - i].data, true, {duration: 1000});
                        }
                        else if (this == s[i] && this.visible == false && s[1 - i].visible == true) {
                            s[i].setVisible(true);
                            s[i].update({dataSorting: {enabled: false, sortKey: 'this.point.stack'}});
                            s[1 - i].update({dataSorting: {enabled: true, sortKey: 'this.point.stack'}});
                            s[1 - i].setData(s[1 - i].data, true, {duration: 1000});
                        }
                        else if (this == s[i] && this.visible == true && s[1 - i].visible == false) {
                        }
                        else if (this == s[i] && this.visible == false && s[1 - i].visible == false) {
                        }
                        console.log("after: " + s[i].name + " - " + s[i].options.dataSorting.enabled);
                    }
                    
                	return false;
                }"""
            }
        }
    },
    
    'chart': {
      'type': 'column',
      'height': 1250,
      'width': 2500
    },
    'legend': {
      'verticalAlign': 'top',
        }
  }

chart.set_dict_options(options)

hero_picks = hero_counts[['hero_name', 'hero_picks']].values.tolist()
hero_bans = hero_counts[['hero_name', 'hero_bans']].values.tolist()

chart.add_data_set(hero_picks, series_type = 'column', name = 'Hero Picks', id = 'Hero Picks', color = 'rgba(56, 171, 48, 0.75)', dataSorting = {'enabled': False})
chart.add_data_set(hero_bans, series_type = 'column', name = 'Hero Bans', id = 'Hero Bans', color = 'rgba(184, 60, 51, 0.75)', dataSorting = {'enabled': False}) 

chart

In [15]:
hero_counts

,hero_id,hero_name,hero_picks,hero_bans,total_pbs
87,129,Mars,30,18,48
49,68,Ancient Apparition,14,33,47
33,46,Templar Assassin,17,25,42
60,88,Nyx Assassin,10,32,42
34,47,Viper,8,25,33
...,...,...,...,...,...
116,93,Slark,0,0,0
117,100,Tusk,0,0,0
118,105,Techies,0,0,0
119,113,Arc Warden,0,0,0


In [22]:
hero_counts.to_csv('C:\\Users\Alec\\OneDrive\\Documents\\Python_Scripts\\hero_counts.csv', index = False)